# Reality Capture & Precision 3D: Introduction to OpenCV in python
#### David Griffiths &nbsp;&nbsp;&nbsp; david.griffiths.16@ucl.ac.uk
#### Jan Boehm &nbsp;&nbsp;&nbsp; j.boehm@ucl.ac.uk

#### Google colab compatability

In [ ]:
# Only run following if using Google Colab
#!git clone https://github.com/3Dimaging-ucl/ucl_cege0092 data

#Only run following if you cannot auto-install requirenments
#install dependencies for OpenCV
#%pip install numpy
#%pip install opencv-python-headless

#Only run following if you cannot install opencv-python-headless
# open terminal (top left menue) and type (without '#')
# sudo apt-get update
# sudo apt-get install libgl1 -y

#### What is an image

In [ ]:
%matplotlib inline
import numpy as np
import pylab as plt

img = np.random.randint(low=0, high=255, size=(100,100, 3), dtype='uint8')
plt.imshow(img), plt.show()

## Basic OpenCV functions

##### Import OpenCV

In [ ]:
import cv2 as cv
# Set image directory '/data/images' for google colab
image_dir = 'images/'

##### Load image

In [ ]:
file = image_dir+'aerial_small.tif'

# Load image in BGR colorspace
img_color = cv.imread(file, cv.IMREAD_COLOR)
#Load image in grayscale
img_gray = cv.imread(file, cv.IMREAD_GRAYSCALE)
# Convert BGR to RGB
img_color_RGB = cv.cvtColor(img_color, cv.COLOR_BGR2RGB)

To plot the images we can create a small helper function. Here we take a list of images, their respective titles, the size of the figure and the subplot shape as input arguments.

In [ ]:
def plot_images(images, titles, figsize, shape, ticks=False):
    ''' Image plotter helper function.
        
        parameters:
            images  : list
            titles  : titles
            figsize : tuple
            shape   : tuple
                      rows x cols
            ticks   : Bool
                      Set True to show axis ticks
        returns:
            None
    '''
    plt.figure(figsize=figsize)
    
    for i in range(len(images)):
        plt.subplot(shape[0],shape[1],i+1),plt.imshow(images[i],'gray')
        plt.title(titles[i])
        if ticks == False:
            plt.xticks([]),plt.yticks([])
    plt.show()
    
    return None

In [ ]:
# Plot images using helper function
plot_images([img_color, img_gray, img_color_RGB], 
            ['BGR Color Image', 'Grayscale Image', 'RGB Color Image'], 
            (20,20),
            (1, 3))

##### Read and edit pixels

OpenCV images are stored in the form of numpy arrays. The numpy array has the dimensions (image height x image width x number of channels). The images can therefore be treated as standard numpy $n$ dimension arrays.

In [ ]:
print ('One channel grayscale image: ', img_gray.shape)
print ('3 Chanel BGR colorspace image: ', img_color.shape)

To access a specific pixel value basic numpy array indexing can be used. For example, to return the intensity of pixel x=500, y= 1000 you can use:

In [ ]:
pixel = 500, 750

print (img_gray[pixel])
print (img_color[pixel])

Notice how the colour image has 3 channels and therefore three pixel intensities (Blue (0), Green (1), Red (2)). Further indexing can reutrn a specific intensity i.e:

In [ ]:
blue = img_color[pixel][0]
green = img_color[pixel][1]
red = img_color[pixel][2]
print (blue, green, red)

**Task**: How would we create a simple 'for' loop to loop through every pixel in a given image?

In [ ]:
# Try task here

**Extra:** How could we use numpy indexing to remove the need for a 'for' loop?

In [ ]:
# Try task here

**Challenge:**

How could we apply the same principles for a RGB image with 3 channels? Load in the image `newyork.jpg` and mask out all pixels except the park using thresholding. Think... what makes these pixels different from the others. 

***Hint*** masked pixels have a value of 0

In [ ]:
img = cv.imread(image_dir+'newyork.jpg', cv.IMREAD_COLOR)
img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
plt.figure(figsize=(20,20)), plt.imshow(img), plt.show()

# Try the rest of the challenge here.

## Geometric Transformations

When carrying out geometric transformations of an image it is important to think of the image as a standard matrix. This way typical linear algebra can be used to perform transformations.

##### Scaling

In [ ]:
# First we will import a new image which which will allow us to easily see the result of the transformations.

img = cv.imread(image_dir+'circle.jpg', 1)
img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
plt.imshow(img)

In [ ]:
# OpenCV offers a resize function which can be called using cv.resize. 
# We will go through a few ways to use this function.

rows, cols, channels = img.shape

# Stretch collumns by a factor of 2
img_resize = cv.resize(img, (cols*2, rows), interpolation = cv.INTER_CUBIC)

# Notice the interpolation method here. OpenCV offers multiple interpolation functions.
# In general it is reccommended:
# Shrinking: cv.INTER_AREA
# Zooming: cv.INTER_CUBIC (slow) or cv.INTER_LINEAR
# Default (general): cv.INTER_LINEAR

# Half the size of the image
img_resize = cv.resize(img, (int(cols/2), int(rows/2)), interpolation = cv.INTER_AREA)

# Set to specified size
img_resize = cv.resize(img, (180, 120),interpolation = cv.INTER_LINEAR)

In [ ]:
plt.imshow(img_resize)

##### Translation

OpenCV provides a function (`cv.warpAffine`) to carry out affine transformations. The main requirements for this are the original image you wish to apply the transformation and the transformation matrix $M$. We must therefore first compute the transformation matrix $M$. If you know the shift in ($x$, $y$) that you wish to compute you can let:  

$M= \begin{bmatrix} 1 & 0 & ∆x \\ 0 & 1 & ∆y \end{bmatrix}$

In [ ]:
M = np.float32([[1,0,25],[0,1,50]])
translation = cv.warpAffine(img, M, (cols, rows))

In [ ]:
plt.imshow(translation)

Similarly a rotation matrix can be computed using an $M$ matrix following:

$M= \begin{bmatrix} \cos(\theta) & -\sin(\theta) \\ \sin(\theta) & \cos(\theta) \end{bmatrix}$

OpenCV provides a tool for creating these matrix with the function `cv.getRotationMatrix2D`

In [ ]:
# Create rotation matrix for 25 degrees rotation in respect to the image center
M = cv.getRotationMatrix2D((cols/2, rows/2), 25, 1)

# Create rotation matrix for 25 degrees rotation with different roation center
M_offset = cv.getRotationMatrix2D((250, 250), 25, 1)

transformation = cv.warpAffine(img, M, (cols, rows))
transformation_offset = cv.warpAffine(img, M_offset, (cols, rows))

print ('M matrix \n', M, '\n\n')
print ('M offset matrix \n', M_offset)

plt.figure(figsize=(7,7))
plt.subplot(221), plt.imshow(transformation)
plt.subplot(222), plt.imshow(transformation_offset)

##### Affine transformation

In affine transformations all parallel lines will remain parallel after the transformation. OpenCV provides a function `cv.getAffineTransformation` to determine the transformation matrix. The minimum number of points for an affine transformation is 3. To determine the $M$ matrix we must therefore make 2 3x1 matrix with the first containing the original co-ordinates of the points, the second will contain the location of the points after the transformation.

In [ ]:
originalPts = np.float32([[50, 50], [100, 150], [180, 120]])
transformedPts = np.float32([[60, 75],[110, 190], [175, 100]])

# Determine M matrix
M = cv.getAffineTransform(originalPts, transformedPts)

# Pass image and M matrix in cv.warpAffine function
transformation = cv.warpAffine(img, M, (cols, rows))

plt.figure(figsize=(7,7))
plt.subplot(221), plt.imshow(img)
plt.subplot(222), plt.imshow(transformation)

##### Perspective transformation

Perspective transformation is similar to affine however instead of preserving parallel lines it preserves collinearity and incidence. As perspective offers an extra degree of translation 4 points are required to determine the $M$ matrix. By providing the co-ordinates in the same manner as the affine transformation, the matrix can be computed using the function `cv.getPerspectiveTransform`. The transormation can then be carried out using `cv.warpPerspective`.

In [ ]:
originalPts = np.float32([[50, 50], [100, 150], [180, 120], [30, 30]])
transformedPts = np.float32([[50, 53], [98, 155], [175, 120], [27, 31]])

# Determine M matrix
M = cv.getPerspectiveTransform(originalPts, transformedPts)

# Pass image and M matrix in cv.warpAffine function
transformation = cv.warpPerspective(img, M, (cols, rows))

plt.figure(figsize=(7,7))
plt.subplot(221), plt.imshow(img)
plt.subplot(222), plt.imshow(transformation)

## Image Thresholding

Basic thresholding in OpenCV is achieved through the `cv.threshold` function. There are multiple options to determine how the threshold can be computed, but first we will create a simple binary threshold. 

**TASK:** Using the code we created earlier how could this be adapted into a simple binary classifier for a grayscale image with a critical value of 150?

In [ ]:
# First lets load and display a digital elevation model (DEM)
img = cv.imread(image_dir+'lidar_small.tif', cv.IMREAD_GRAYSCALE)
plt.imshow(img, 'gray')

In [ ]:
# Try task here

**Extra**: How could we use numpy indexing to remove the need for a 'for' loop?

In [ ]:
# Try task here

We will now carry this out using OpenCV. The function cv.THRESH_BINARY defines the threshold style. The availble styles are:

`cv.THRESH_BINARY`

`cv.THRESH_BINARY_INV`

`cv.THRESH_TRUNC`

`cv.THRESH_TOZERO`

`cv.THRESH_TOZERO_INV`

In [ ]:
ret, thresh = cv.threshold(img, 50, 255, cv.THRESH_BINARY)
ret, thresh_inv = cv.threshold(img, 50, 255, cv.THRESH_BINARY_INV)

plot_images([img, thresh, thresh_inv],
            ['Original', 'Binary', 'Binary Inverse'],
            (15,15),
            (1, 3))

In [ ]:
adaptive_thresh = cv.adaptiveThreshold(img,255, \
                            cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY,15,7)

adaptive_thresh_g = cv.adaptiveThreshold(img,255, \
                                cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY,15,7)

plot_images([adaptive_thresh, adaptive_thresh_g],
            ['Adaptive Thresh', 'Adaptive Thresh w/ Gaussian'],
            (10,10),
            (1, 2))

##### Otsu's Binarisation

Otsu's binirisation is a method to automatically determine the threshold for a bimodal image. This attempts to find the optimum threshold value by converting the image to a histogram and finding the value in between the two main peaks. This method is therefore only suitable for bimodal images.

In [ ]:
ret, otsu = cv.threshold(img,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)

plot_images([img, thresh, adaptive_thresh, otsu],
            ['Original', 'Binary Threshold', 'Adaptive Threshold', 'Otsu Binarisation'],
            (20, 20),
            (1, 4))


## Convolution Filters

<img src="images/conv2d_padding.gif">

Convolutional filters processes images by passing a matrix (kernel) over each pixel within an image. The dot product of the weighted kernel matrix and the overlaying pixels of the image is used to create a new pixel value at the given point. Each pass is often referred to as a convolution.  This can be used for common image processing such as blurring, sharpening and edge detection. OpenCV provides many pre-built functions for kernels convolutions as well as the `cv.filter2D()` function which can be used to pass a custom-made filter over an image.

### Bluring

Bluring (or smoothing) is typically used in image processing for noise reduction and smoothing. Here we achieve this by convolving a low-pass filter kerel over an image.

##### Basic Blurring

In [ ]:
img = cv.imread(image_dir+'noise.jpg', 1)
img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
img_gs = cv.imread(image_dir+'noise.jpg', 0)

# A simple averaging filter could be made by passing first creating a kerel
kernel = np.ones((5,5), np.float32)/25
# Using the filter2D function the image can be averaged
averaged_img = cv.filter2D(img, -1, kernel)

#Alternatively we can use a pre-made OpenCV blur
mean_blur = cv.blur(img_gs,(5,5))

#Median blurs are good for removing salt and pepper effects

img_sp = cv.imread(image_dir+'salt.jpg', 0)

median_blur = cv.medianBlur(img_sp, 5)

plot_images([img, mean_blur, img_sp, median_blur],
            ['Cat Image', 'Mean Blur', 'Salt and Pepper Image', 'Median Blur'],
            (10, 10),
            (2, 2))

##### Gaussian Blurring

The gaussian blur uses the gaussian function to calculate each pixels transformation. The equation used to define the Guassian function in 2-dimensions ($x$ and $y$) is defined as: $G_{(x)} = \frac{1}{\sqrt{2\Pi\sigma^2}}e^{-\frac{x^2}{2\sigma^2}}$. 

The resulting 3x3 kernel used is: $G~(x,y)= \begin{bmatrix} 1 & 2 & 1 \\ 2 & 4 & 2 \\ 1 & 2 & 1 \end{bmatrix} * \frac{1}{16}$

As blurring filters smooth areas with a high variance of intensity they have a strong response to edges. This results in the smoothing of an edge. If our image processing problem is concerned with the edges this is not ideal. Bilateral filters preserve edges by considering the neighbouring pixels and weights assainged to them. The weights have two components, the first is the same weighting as used in the Gaussian kernel above. The second takes into account the difference in intensity between the neighbouring pixels and the evaluated one. A more detailed description of this can be [found here](http://homepages.inf.ed.ac.uk/rbf/CVonline/LOCAL_COPIES/MANDUCHI1/Bilateral_Filtering.html).

If we are primarilly looking at denoising, OpenCV offers more specialised features for such tasks. Take a look at `cv.fastNlMeansDenoising()` and `cv.fastNlMeansDenoisingColored()` if you are dealing with images that contain a lot of noise.

In [ ]:
gaus_blur = cv.GaussianBlur(img, (7, 7), 0)

plot_images([img, gaus_blur],
            ['Original', 'Gaussian Blurred'],
            (15, 15),
            (1, 2))

### Edge Detection and Image Gradients

Edge detection is one of the most fundamental operations used in image processing. Edge detection can be used for feature extraction/segmentation as well as limiting operations to pixels with geometric interest. Here we will look at some of the most common edge detectors discussed in this module.

<img src="images/edge_detection2.png" >

##### Sobel Operator 

The sobel operator is a kernal convolution that is used to find edges within an image. The sobel operator is a discrete differentiation operator which computes an approximation to the derivative of an image that is serperate in the x and y direction. This therefore responds strongly to areas where there is a high variation of pixel intensities. By default this is achieved by passing the following 3x3 kernels over a given image:

$G~x= \begin{bmatrix} -1 & 0 & 1 \\ -2 & 0 & 2 \\ -1 & 0 & 1 \end{bmatrix}$ &nbsp;&nbsp;&nbsp; $G~y= \begin{bmatrix} -1 & -2 & -1 \\ 0 & 0 & 0 \\ 1 & 2 & 1 \end{bmatrix}$

Using the kernel size argument '`ksize`' the size of the kernel can be increased. If `ksize=-1` the Sharr derivative is used instead of the sobel. When using a 3x3 kernel size OpenCV reccommends using the Sharr kernal which is as follows:

$G~x= \begin{bmatrix} -3 & 0 & 3 \\ -10 & 0 & 10 \\ -3 & 0 & 3 \end{bmatrix}$ &nbsp;&nbsp;&nbsp; $G~y= \begin{bmatrix} -3 & -10 & -3 \\ 0 & 0 & 0 \\ 3 & 10 & 3 \end{bmatrix}$

In [ ]:
file = image_dir+'lidar_small.tif'
img = cv.imread(file, 0)

sobelx64f = cv.Sobel(img,cv.CV_64F,1,0,ksize=3)
abs_sobel64f = np.absolute(sobelx64f)
sobelx_8u = np.uint8(abs_sobel64f)

sobely64f = cv.Sobel(img,cv.CV_64F,0,1,ksize=3)
abs_sobel64f = np.absolute(sobely64f)
sobely_8u = np.uint8(abs_sobel64f)

plot_images([img, sobelx_8u, sobely_8u],
            ['Original', 'Sobel X', 'Sobel Y'],
            (20, 20),
            (1, 3))

Whilst the sobel is useful for finding edges in the $x$ and $y$ directions independently, rarely would we use these as our final output. It would be more common to combine the two images which contains all edges. The below function shows us how given an image as input we can return an edge magnitude image for both $x$ any $y$ directions.

In [ ]:
def getGradientMagnitude(im):
    ddepth = cv.CV_32F
    dx = cv.Sobel(im, ddepth, 1, 0)
    dy = cv.Sobel(im, ddepth, 0, 1)
    dxabs = cv.convertScaleAbs(dx)
    dyabs = cv.convertScaleAbs(dy)
    mag = cv.addWeighted(dxabs, 0.5, dyabs, 0.5, 0)
    return mag

In [ ]:
# OpenCV offer a function to return the kernel coefficients

sobel7x_deriv = cv.getDerivKernels(1, 0, 5)
sobel7y_deriv = cv.getDerivKernels(0, 1, 5)

# To display this we need to compute the outer product of the two vectors

print ('Sobel 7x7 kernel x direction:\n', np.outer(sobel7x_deriv[1], sobel7x_deriv[0]))
print ('\nSobel 7x7 kernel y direction:\n', np.outer(sobel7y_deriv[1], sobel7y_deriv[0]))

##### Laplacian Edge Detector

The laplacian edge detector computes the residual of an image using the equation, $\frac{\partial^2src}{\partial x^2} + \frac{\partial^2src}{\partial y^2}$ where the derivatives for $x$ and $y$ are found using sobel in the $x$ and $y$ direction respectively. Due to this relationship when we call `cv.Laplacian` the sobel operator is run internally. When the kernel size `ksize=1` the following filter is applied.

$G= \begin{bmatrix} 0 & 1 & 0 \\ 1 & -4 & 1 \\ 0 & 1 & 0 \end{bmatrix}$

Alternatively we could also adapt this to consider diagonals using the following:

$G~x= \begin{bmatrix} 1 & 1 & 1 \\ 1 & -8 & 1 \\ 1 & 1 & 1 \end{bmatrix}$

In [ ]:
# Scaling can be used to scale the returned pixel intensity

laplacian = cv.Laplacian(img, cv.CV_8U, ksize=1, scale=1) 
abs_lap64f = np.absolute(laplacian)
lap_8u = np.uint8(abs_lap64f)

laplacian_s5 = cv.Laplacian(img, cv.CV_8U, ksize=1, scale=5)
abs_lap64f_s5 = np.absolute(laplacian_s5)
lap_8u_s5 = np.uint8(abs_lap64f_s5)

plot_images([lap_8u, lap_8u_s5],
            ['Laplacian', 'Laplacian Scales x5'],
            (15, 15),
            (1, 2))

##### Canny Edge Detector

Developed by John F. Canny in 1986, the canny edge detection algorithms extracts edges using a 5 stage approach:

* Noise removal and smoothing using Gaussian filter
* Gradient intensity representations for the image computed (Sobel)
* Apply non-maximum suppression to remove false edges
* Thresholding by applying an upper and lower boundary on the gradient values
* Hysteresis thresholding to track along connected edges

In OpenCV all of these steps are wrapped into 1 function `cv.Canny()`. The function takes 4 main arguments:

* Lower threshold for hysteresis thresholding (Double)
* Upper threshold for hysteresis thresholding (Double)
* Kernel size for sobel operator (Int)
* Whether or not to use $L_{2}$ norm for image gradient instead of the default $L_{1}$ (Bool)

In [ ]:
canny_lowthresh = cv.Canny(img, 25,75,5,L2gradient=True)
canny_highthresh = cv.Canny(img,100,250,5,L2gradient=True)

plot_images([canny_lowthresh, canny_highthresh],
            ['Low threshold', 'High threshold'],
            (15, 15),
            (1, 2))

### Morphological Filters

The last topic we will look at is morphological transformations. As the name suggests we are going to make physical changes to the shape of images. This is usually performed on grayscale images although it is possible to carry this out on RGB images if appropriate measures are taken. The most common forms of morphological transformations are:

* Dilation
* Erosion
* Opening
* Closing

However, OpenCV offers support for a larger number of transformations. Also, take a look at [scikit-image](http://scikit-image.org) where you can find many unique and novel transformations. Here we will cover the use of the erosion filter to remove non-terrain objects (i.e. buildings, vegetation, cars etc.) from Digital Surface Models (DSMs) to generate Digital Terrain Models (DTMs). A good introduction to all the filters mentioned above can be found [here](https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_imgproc/py_morphological_ops/py_morphological_ops.html).

In [ ]:
file = image_dir+'lidar_small.tif'
img = cv.imread(file, 0)

# Define our erosion kernel
# Task: Find a suitable kernel (structure element) size
kernel = np.ones((3,3),np.uint8)

# Perform erosion
erosion = cv.erode(img,kernel,iterations = 1)

# Smooth resulting DTM with a Gaussian Blur
erosion_g = cv.GaussianBlur(erosion, (3, 3), 1)

# Alternatively we could using the dilation operator on the eroded
# image to remove any small remaining above ground artifacts. This
# is very similar to the 'Opening' operator

opening = cv.dilate(erosion,kernel,iterations = 1)

plot_images([img, erosion_g, opening], 
            ['Original', 'Erosion', 'Opening = Erosion->Dilation'], 
            (10,10), 
            (1, 3))

Once we have a satisfactory DTM we can then use this to remove the terrain from the image returning only the above ground objects. We can achieve this by using the `cv.subtract()` or `cv.absdiff()` functions. As the difference will never be perfect (i.e. all - ground = above) we are usually left will a very small value for the ground. To remove this we can either perform a manual threshold, or alternatively, use an otsu binarisation. Otsu is a good model here as the image will likely be bimodal (ground and above ground).

In [ ]:
# Subtract DTM from DSM
# Task: compare applying this to erosion vs opening (erosion + dilation)
res = cv.subtract(img, erosion)

# Otsu binarisation
ret, otsu = cv.threshold(res,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)

plot_images([img, res, otsu], 
            ['Original', 'Subtracted', 'Otsu'], 
            (20,20), 
            (1, 3))

### Extras...

### Contours

Until now we have been carrying out segmentation of images but have not had any real way of handling the segmented pixels. The easiest way to handle an extracted feature is to find the contours of the image. Contours are just a curve that connects continous points along a given boundary. To accurately find contours in OpenCV we need to first convert the image into a binary image. This can be achieved using the techniques discussed above (i.e. thresholding, edge detection and morphological filters). By using these tools we create a white feature of interest on a black background, if this is achieved contours can be easily drawn along these boundaries (preferrably white objects on black background). OpenCV returns countours in a list of coordinates that describe the shape of the contour it represents. If the contour is large the list will be large. To reduce this a contour approximation method can be used which only stores the important points (i.e. direction edges). If the contour is a very complicated shape this list may still be very long. This is very common when the contour line is derived from an edge detector and is very noisy. To correct for this we can apply a contour approximation which fits the contour to a similar (and more simple) shape (see the **Douglas-Peucker** algorithm for more information).

Let's take a look at how we can extract the contours for all the above-ground objects we just created, and draw them back onto the original image.

In [ ]:
# Find contours
image, contours, hierarchy = cv.findContours(otsu, cv.RETR_TREE,cv.CHAIN_APPROX_SIMPLE)

# Draw contours onto image
original_img = img.copy()
img = cv.cvtColor(img, cv.COLOR_GRAY2RGB)
img = cv.drawContours(img, contours, -1, (0, 0, 255), 3)

plot_images([original_img, img], ['Original', 'Contour Image'], (20,20), (1,2))